In [42]:
import pandas as pd

file_path = "./data/가구인테리어_validation.csv"

df = pd.read_csv(file_path, nrows=5000, low_memory=False)

In [43]:
df.head()

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,c,러그 식이네요,가구인테리어,211,q,m,제품_일반_비교,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,러그,791,431
1,2,s,네,가구인테리어,211,a,m,제품_일반_비교,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,791,432
2,3,c,아까거랑 뭐가 달라요?,가구인테리어,10,q,m,제품_일반_비교,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4353,21
3,4,s,아까는 60수였어요 여긴 40수,가구인테리어,10,a,m,제품_일반_비교,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4353,22
4,5,c,"아 요것도 괜찮고, 가격 면에서는 저것도 괜찮고",가구인테리어,50,q,m,제품_일반_비교,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4791,98


In [44]:
# 그룹별 row 수 계산
group_sizes = df.groupby('상담번호').size()

# row 수가 2인 그룹의 상담번호만 필터링
valid_ids = group_sizes[group_sizes == 2].index

# row 수가 2인 그룹만 필터링하여 grouped_data 생성
grouped_data = []
for idx, group in df[df['상담번호'].isin(valid_ids)].groupby('상담번호'):
    group_dict = {
        "Q": group.loc[group['QA여부'] == 'q', '발화문'].tolist(),
        "A": group.loc[group['QA여부'] == 'a', '발화문'].tolist(),
        "id": idx
    }
    # 리스트 형태로 변환된 데이터만 추가
    group_dict['Q'] = ' '.join(group_dict['Q'])  # 각 질문을 하나의 문자열로 결합
    group_dict['A'] = ' '.join(group_dict['A'])  # 각 답변을 하나의 문자열로 결합
    grouped_data.append(group_dict)

# 딕셔너리 리스트를 DataFrame으로 변환
grouped_df = pd.DataFrame(grouped_data)

In [45]:
grouped_df[:5]

,Q,A,id
0,2셋트시키면 4장이 오는 거 맞나요?,2장 구성 상품으로 수량 2개 주문시 총 4장으로 발송되니 참고 부탁드리겠습니다.,64
1,두 장오는 거 맞나요?,두 장 구성 상품이며 옵션 선택 시 색상/색상표기되어 있으니 참고 부탁드리겠습니다.,71
2,요제품 1+1 아닌가요?,해당 쇼핑몰에서 주문해 주시면 1매 증 정품으로 함께 배송되오니 주문 시 참고 부탁...,80
3,지금 구입하면 타이백주나요?,구성 확인 부탁드리겠습니다. 다른 문의사항은 언제든 글 남겨주시구요.,122
4,스냅은 상품당 3개 오는 건가요?,블라인드 상단 브라켓은 사이즈에 맞는 수량으로 배송됩니다 브라켓은 나사로 설치해 주...,280


In [ ]:
# 데이터셋을 CSV 파일로 저장
csv_filename = "makedata.csv"
grouped_df.to_csv(csv_filename, index=False, encoding="utf-8")

## 이미 있는 데이터셋을 토크나이즈 하고 훈련하는 부분

In [5]:
from transformers import (
    GPT2LMHeadModel, # koGPT2 모델 로드할 때 필요한 모듈
    PreTrainedTokenizerFast, # koGPT2용으로 만들어진 토크나이저 로드할 때 필요한 모듈
    Trainer, # Huggingface 의 전용 모델 트레이닝 모듈
    TrainingArguments, 
    DataCollatorForLanguageModeling,
)
import torch
from datasets import Dataset
import pandas as pd

In [6]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
MASK = "<unused0>"
SENT = "<unused1>"
PAD = "<pad>"

In [7]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(  # koGPT2를 사용하는데 필요한 토크나이저 정의
    "skt/kogpt2-base-v2",
    bos_token=BOS,
    eos_token=EOS,
    unk_token="<unk>",
    pad_token=PAD,
    mask_token=MASK,
)

additional_special_tokens = [Q_TKN, A_TKN, SENT]
tokenizer.add_special_tokens({'additional_special_tokens': additional_special_tokens})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


0

In [35]:
csv_path = "AUG/data_rd.csv"
qa_df = pd.read_csv(csv_path)

In [36]:
qa_df

,Q,A,id
0,2셋트시키면 4장이 오는 거 맞나요?,2장 구성 상품으로 수량 2개 주문시 총 4장으로 발송되니 참고 부탁드리겠습니다.,64
1,두 장오는 거 맞나요?,두 장 구성 상품이며 옵션 선택 시 색상/색상표기되어 있으니 참고 부탁드리겠습니다.,71
2,요제품 1+1 아닌가요?,해당 쇼핑몰에서 주문해 주시면 1매 증 정품으로 함께 배송되오니 주문 시 참고 부탁...,80
3,지금 구입하면 타이백주나요?,구성 확인 부탁드리겠습니다. 다른 문의사항은 언제든 글 남겨주시구요.,122
4,스냅은 상품당 3개 오는 건가요?,블라인드 상단 브라켓은 사이즈에 맞는 수량으로 배송됩니다 브라켓은 나사로 설치해 주...,280
...,...,...,...
9657,개 구매했으면 이니까 총 개와야 하는 거,사은품 추가 출고로 진행도와 드리겠습니다. 문의 남겨주셔서 감사합니다.,ri-308306
9658,두 주문했으면 동일 제품 한 개 추가로 거 맞죠,주문 내역 확인하였으며 총 3개 배송됩니다.,ri-308313
9659,커블체어와이드 무료,이벤트 기간 내 주문건으로 무료 반품 가능하며 판매자 송금으로 반품 접수해주시면 됩니다.,ri-308320
9660,배송비 입금해 문자가,입금 확인되어 반품 완료되었습니다.,ri-308322


In [37]:
# 2. Q&A 데이터를 포맷팅 및 토크나이즈
def preprocess_data(examples):
    """
    각 데이터에 대해 (BOS) + (Q_TKN) + 질문 + (SENT) + (A_TKN) + 답변 + (EOS) 포맷으로 토크나이즈
    """
    formatted_texts = [
        f"{BOS} {Q_TKN} {q} {A_TKN} {a} {EOS}"
        for q, a in zip(examples["Q"], examples["A"])
    ]
    tokenized = tokenizer(
        formatted_texts,
        truncation=True,
        padding="max_length",
        max_length=64
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # labels는 input_ids와 동일
    return tokenized

In [38]:
# Dataset 생성
raw_dataset = Dataset.from_dict(qa_df)

# 토크나이즈 적용
tokenized_dataset = raw_dataset.map(preprocess_data, batched=True);

len(tokenized_dataset)

Map:   0%|          | 0/9662 [00:00<?, ? examples/s]

9662

In [39]:
len(tokenized_dataset)

9662

In [40]:
decoded_text = tokenizer.decode(tokenized_dataset[4840]["input_ids"], skip_special_tokens=False)
print(decoded_text)

</s> <usr> 개가 온다는 맞는 건가요 <sys> 주문하신 상품과 같은 상품으로 배송됩니다. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [41]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd

# (이미 Q&A 데이터프레임인 qa_df를 갖고 있다고 가정)
# raw_dataset = Dataset.from_dict(qa_df)
# tokenized_dataset = raw_dataset.map(preprocess_data, batched=True)
# 위 과정은 이미 진행되었다고 가정
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 훈련/평가 데이터 분리 (예: 90%/10%)
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)

train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 모델 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.to(device)  # 모델을 GPU로 이동

# 만약 Q_TKN, A_TKN 등을 추가하였다면 토크나이저의 vocab 크기가 달라졌을 수 있으므로
# 아래와 같이 모델 임베딩 사이즈를 재조정 필요
model.resize_token_embeddings(len(tokenizer))

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./AUG_RD_MODEL",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",  # 에폭마다 평가
    save_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    logging_steps=100,
    save_total_limit=10,
    fp16=True,
    report_to="none"  # Weights & Biases 등의 로깅 툴을 사용하지 않을 경우 none으로 설정
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# 학습 시작
trainer.train()

del model
del trainer
torch.cuda.empty_cache()

/home/team_user/miniconda3/envs/learningtest/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_253086/195463389.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,2.044900,1.867075
2,1.655200,1.707879
3,1.402000,1.587788
4,1.203100,1.494536
5,1.039400,1.417817
6,0.898600,1.362488
7,0.801900,1.320492
8,0.718600,1.292292
9,0.664000,1.273671
10,0.625700,1.268106
